In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
#from google.colab.patches import cv2_imshow
from PIL import Image
import os

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
def mean_pixels(image, index, n):
    start_row, start_col = index[0] - n, index[1] - n
    end_row, end_col = index[0] + n + 1, index[1] + n + 1
    row_indices = np.clip(range(start_row, end_row), 0, image.shape[0] - 1)
    col_indices = np.clip(range(start_col, end_col), 0, image.shape[1] - 1)
    masked_image = image[np.ix_(row_indices, col_indices)]
    mean_values = np.mean(np.mean(masked_image, axis=0), axis=0)

    n_val=int(n/10)
    start_row1, start_col1 = index[0] - n_val, index[1] - n_val
    end_row1, end_col1 = index[0] + n_val + 1, index[1] + n_val + 1
    row_indices1 = np.clip(range(start_row1, end_row1), 0, image.shape[0] - 1)
    col_indices1 = np.clip(range(start_col1, end_col1), 0, image.shape[1] - 1)
    for i in range(len(row_indices1)):
      image[row_indices1[i],col_indices1[i]]=mean_values


In [ ]:
yourpath="C:/Users/Kompikl/Downloads/CESSA/CESSA"
path_to_save='C:/Users/Kompikl/Downloads/mgr'
for root, dirs, files in os.walk(yourpath, topdown=False):
    for name in files:
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".jpg":
          img = cv2.imread(os.path.join(root, name))

          rows, cols, ch = img.shape
          # Mask dimensions to mean pixels
          neigh_size=int(min(rows,cols)/175.0)

          # Convert to grayscale
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
          # Threshold the image to binarize it
          
          threshold_value, binary_img = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
          #threshold_value, binary_img = cv2.threshold(gray, tr, 255, cv2.THRESH_BINARY) 

          nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_img, None, None, None, 8, cv2.CV_32S)

          
          #get CC_STAT_AREA component as stats[label, COLUMN] 
          areas = stats[1:,cv2.CC_STAT_AREA]

          result = np.zeros((labels.shape), np.uint8)

          for i in range(0, nlabels - 1):
              if areas[i] >= 140:   #keep
                  result[labels == i + 1] = 255

          result = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)

          # Image reconstruction

          #Replace non black pixels from binary image as original image pixels
          cond = img> 128 #200-255
          pixels= np.where(cond,result,img)
            
          cond = result==255 
          pixels=np.where(cond,img,pixels )
            
          #Mean the image regions, where black pixels was
          indices=np.where(pixels==0)
          for i in range(len(indices[0])):
            mean_pixels(pixels, (indices[0][i],indices[1][i]),neigh_size)


          noiseless_image_colored = cv2.fastNlMeansDenoisingColored(pixels,None,15,15,7,21) 
          cv2.imwrite(os.path.join(path_to_save , name),noiseless_image_colored)

          